<a href="https://colab.research.google.com/github/aetev/Hearth-Stone-Python-Simulator/blob/main/card_enc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [142]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [143]:
import json
import tensorflow as tf
import re
from tensorflow.keras.layers import Embedding
# Assuming AtomicCards.json is in your Google Drive, adjust the path if necessary
file_path = '/content/drive/MyDrive/MTGdata/AtomicCards.json'


with open(file_path, 'r') as f:
  data = json.load(f)['data']

In [ ]:
def filter_legal_cards(data):
    legal_commander_data = {}
    for card_name, card_data in data.items():
        try:
            if card_data[0]['legalities']['commander'] == 'Legal':
                legal_commander_data[card_name] = card_data[0] # Store only the first element of card_data
        except (KeyError, IndexError):
            pass  # Or print a message, log it, etc.
    return legal_commander_data

filtered_data = filter_legal_cards(data)


In [ ]:
items = list(filtered_data.items())

second_key, second_value = items[11]

text = second_value['text']
print(text)

In [ ]:
class TextCleaningLayer(tf.keras.layers.Layer):
    def __init__(self,sentence_list, **kwargs):
        super(TextCleaningLayer, self).__init__(**kwargs)
        self.vectorizer = TextVectorization(standardize=None, ngrams=2, output_mode='int',output_sequence_length=150)
        self.vectorizer.adapt(cleaned_text(sentence_list))
        self.embedding_layer = Embedding(input_dim=50, output_dim=10)

    def clean_text(self, text):
        # Convert input to string tensor if it's not already
        if not isinstance(text, tf.Tensor):
            text = tf.convert_to_tensor(text, dtype=tf.string)

        cleaned_text = tf.strings.lower(text)
        cleaned_text = tf.strings.regex_replace(cleaned_text, '\n', '')
        cleaned_text = tf.strings.regex_replace(cleaned_text, '\.', ' ')
        cleaned_text = tf.strings.regex_replace(cleaned_text, '\{|\}', ' ')
        cleaned_text = tf.strings.regex_replace(cleaned_text, '\([^)]*\)', '')
        cleaned_text = tf.strings.regex_replace(cleaned_text, '[^\w\s+/\-]', '')
        cleaned_text = tf.strings.regex_replace(cleaned_text, '\s+', ' ')
        return cleaned_text

    def call(self, inputs):
        cleaned_text = self.clean_text(inputs)
        vectorized_text = self.vectorizer(cleaned_text)
        embedded_text = self.embedding_layer(vectorized_text)
        return embedded_text

# Example usage with a batch of texts
batch_texts = tf.constant([text])
cleaned_text_layer = TextCleaningLayer(batch_texts)
cleaned_batch = cleaned_text_layer.call(batch_texts)

for word in cleaned_batch[0]:
  print(word)

